In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [6]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [7]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [8]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "30E0VCwGv5VuuyolQrLFxWAec"
    consumer_secret = "GomlqU48d3zajUyIPHkQhVRfY5MCJRxIA0g30gR2OQy1k69OEA"
    access_token = "1517731942778556416-DDqmzAYnH0wNRRU7BOdOVF8n96GDyc"
    access_token_secret = "AeXjItkIt56JQTOitaEUDchPT9J4wNXNw6eFVNupqjN72"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1518754516891430912, 'username': 'PhillipNakov', 'created_at': 'Tue Apr 26 00:51:08 +0000 2022', 'lng': -118.325145, 'lat': 34.182159999999996, 'text': 'And how will they ever A) actually get him to pay said fines? B) how will he ever be compelled to pay or do anythin… https://t.co/0ACpqm3K5O'}
{'id': 1518754517117980672, 'username': 'MaheshNYCTO', 'created_at': 'Tue Apr 26 00:51:08 +0000 2022', 'lng': -73.9487755, 'lat': 40.655138, 'text': 'The 3 point shooting…'}
{'id': 1518754517403140096, 'username': 'PIMPonWALLST', 'created_at': 'Tue Apr 26 00:51:08 +0000 2022', 'lng': -75.117998, 'lat': 40.0048655, 'text': 'Harden need a mid range game, he can’t blow by anybody anymore'}
{'id': 1518754518019706880, 'username': 'CJKChel', 'created_at': 'Tue Apr 26 00:51:08 +0000 2022', 'lng': -112.12467649999999, 'lat': 33.5528625, 'text': 'If you haven’t seen the new episode from the @QuackReportPod, check it out! Support these good dudes.📺 -… https://t.co/5d2jowXbrg'}
{'id': 1518754518565

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
